
# Basic linear algebra in torchTT
</p>
This notebook is an introduction into the basic linar algebra operations that can be perfromed using the torchtt package.
The basic operations such as +,-,*,@,norm,dot product can be performed between torchtt.TT instances without computing the full format by computing the TT cores of the result.
One exception is the elementwise division between TT objects. For this, no explicit form of the resulting TT cores can be derived and therefore optimization techniques have to be employed (see the notebook fast_tt_operations.ipynb).

Imports

In [ ]:
import torch as tn
try:
    import torchtt as tntt
except:
    print('Installing torchTT...')
    %pip install git+https://github.com/ion-g-ion/torchTT

We will create a couple of tensors for the opperations that follow

In [ ]:
N = [10,10,10,10]
o = tntt.ones(N)
x = tntt.randn(N,[1,4,4,4,1])
y = tntt.TT(tn.reshape(tn.arange(N[0]*N[1]*N[2]*N[3], dtype = tn.float64),N))
A = tntt.randn([(n,n) for n in N],[1,2,3,4,1])
B = tntt.randn([(n,n) for n in N],[1,2,3,4,1])

### Addition

The TT class has the "+" operator implemeted. It performs the addition between TT objects (must have compatible shape and type) and it returns a TT object. 
One can also add scalars to a TT object (float/int/torch.tensor with 1d).

The TT rank of the result is the sum of the ranks of the inputs. This is usually an overshoot and rounding can decrease the rank while maintaining the accuracy.

Here are a few examples:

In [ ]:
z = x+y 
print(z)
# adding scalars is also possible
z = 1+x+1.0
z = z+tn.tensor(1.0)
# it works for the TT amtrices too
M = A+A+1 
print(M)

### Subtraction



### Multiplication (elementwise)

One can perform the elementwise multiplication $\mathsf{u}_{i_1...i_d} = \mathsf{x}_{i_1...i_d} \mathsf{y}_{i_1...i_d}$ between 2 tensors in the TT format without goin to full format.
The main issues of this is that the rank of the result is the product of the ranks of the input TT tensors.

In [ ]:
u = x*y
print(u)

M2 = A*A

### Matrix vector product and matrix matrix product

* TT matrix and TT tensor: $(\mathsf{Ax})_{i_1...i_d} = \sum\limits_{j_1...j_d}\mathsf{A}_{i_1...i_d,j_1...j_d} \mathsf{x}_{j_1...j_d}$
* TT matrix and TT matrix: $(\mathsf{AB})_{i_1...i_d,k_1...k_d} = \sum\limits_{j_1...j_d}\mathsf{A}_{i_1...i_d,j_1...j_d} \mathsf{B}_{j_1...j_d,k_1...k_d}$

Multiplication can be performed between a TT operator and a full tensor (in torch.tensor format) the result in this case is a full tn.tensor

In [ ]:
print(A@tn.rand(A.N, dtype = tn.float64))

### Kronecker product


For computing the Kronecker product one can either use the "**" operator or the method torchtt.kron().

In [ ]:
print(x**y)
print(A**A)

### Norm

Frobenius norm of a tensor $||\mathsf{x}||_F^2 = \sum\limits_{i_1,...,i_d} \mathsf{x}_{i_1...i_d}$ can be directly domputed from a TT decomposition.

In [ ]:
print(y.norm())
print(A.norm())

### Dot product and summing along modes

### Reshaping

### Multiple operations